In [1]:
!pip3 install duckdb

Defaulting to user installation because normal site-packages is not writeable


In [2]:
import duckdb

In [3]:
# Checks calendar.csv
duckdb.sql("""SELECT * FROM calendar.csv""").df().head(5)

,DATE,DAY_OF_WEEK,TEXT_DAY_OF_WEEK,TEXT_MONTH_OF_YEAR,IS_HOLIDAY,IS_WORKING_DAY
0,1900-01-01,1,Monday,January,True,True
1,1900-01-02,2,Tuesday,January,False,True
2,1900-01-03,3,Wednesday,January,False,True
3,1900-01-04,4,Thursday,January,False,True
4,1900-01-05,5,Friday,January,False,True


In [4]:
# Checks data.csv
duckdb.sql("""SELECT * FROM data.csv""").df().head(5)

,FUND_CODE,MARKET_DATE,RETUNR_TYPE,DAILY_RETURN
0,FUND_01,2025-01-01,TYPE_A,0.300730
1,FUND_01,2025-01-02,TYPE_A,0.148510
2,FUND_01,2025-01-03,TYPE_A,0.043051
3,FUND_01,2025-01-04,TYPE_A,NaN
4,FUND_01,2025-01-05,TYPE_A,NaN


In [5]:
# In-memory database
con = duckdb.connect(database=':memory:')

In [6]:
# Creates tables / Rename cols
con.execute("CREATE TABLE IF NOT EXISTS tb_calendar AS FROM calendar.csv")
con.execute("CREATE TABLE IF NOT EXISTS tb_market_data AS FROM data.csv")
con.execute("ALTER TABLE tb_market_data RENAME COLUMN RETUNR_TYPE TO RETURN_TYPE")

In [7]:
# Checks tb_calender structure
con.sql("DESCRIBE tb_calendar")

┌────────────────────┬─────────────┬─────────┬─────────┬─────────┬─────────┐
│    column_name     │ column_type │  null   │   key   │ default │  extra  │
│      varchar       │   varchar   │ varchar │ varchar │ varchar │ varchar │
├────────────────────┼─────────────┼─────────┼─────────┼─────────┼─────────┤
│ DATE               │ DATE        │ YES     │ NULL    │ NULL    │ NULL    │
│ DAY_OF_WEEK        │ BIGINT      │ YES     │ NULL    │ NULL    │ NULL    │
│ TEXT_DAY_OF_WEEK   │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ TEXT_MONTH_OF_YEAR │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ IS_HOLIDAY         │ BOOLEAN     │ YES     │ NULL    │ NULL    │ NULL    │
│ IS_WORKING_DAY     │ BOOLEAN     │ YES     │ NULL    │ NULL    │ NULL    │
└────────────────────┴─────────────┴─────────┴─────────┴─────────┴─────────┘

In [8]:
# Checks tb_market_data structure
con.sql("DESCRIBE tb_market_data")

┌──────────────┬─────────────┬─────────┬─────────┬─────────┬─────────┐
│ column_name  │ column_type │  null   │   key   │ default │  extra  │
│   varchar    │   varchar   │ varchar │ varchar │ varchar │ varchar │
├──────────────┼─────────────┼─────────┼─────────┼─────────┼─────────┤
│ FUND_CODE    │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ MARKET_DATE  │ DATE        │ YES     │ NULL    │ NULL    │ NULL    │
│ RETURN_TYPE  │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ DAILY_RETURN │ DOUBLE      │ YES     │ NULL    │ NULL    │ NULL    │
└──────────────┴─────────────┴─────────┴─────────┴─────────┴─────────┘

In [9]:
# Checks tb_calendar content
con.sql("""SELECT * FROM tb_calendar""").df().head(5)

,DATE,DAY_OF_WEEK,TEXT_DAY_OF_WEEK,TEXT_MONTH_OF_YEAR,IS_HOLIDAY,IS_WORKING_DAY
0,1900-01-01,1,Monday,January,True,True
1,1900-01-02,2,Tuesday,January,False,True
2,1900-01-03,3,Wednesday,January,False,True
3,1900-01-04,4,Thursday,January,False,True
4,1900-01-05,5,Friday,January,False,True


In [10]:
# Checks tb_market_data content
con.sql("""SELECT * FROM tb_market_data""").df().head(5)

,FUND_CODE,MARKET_DATE,RETURN_TYPE,DAILY_RETURN
0,FUND_01,2025-01-01,TYPE_A,0.300730
1,FUND_01,2025-01-02,TYPE_A,0.148510
2,FUND_01,2025-01-03,TYPE_A,0.043051
3,FUND_01,2025-01-04,TYPE_A,NaN
4,FUND_01,2025-01-05,TYPE_A,NaN


In [11]:
# Exploratory Data Analysis for electing a good unique business-key
# In this case, fund_code + market_date is not ideal because it's not unique
con.sql("""
    SELECT COUNT(1), fund_code, market_date, AVG(daily_return)
      FROM tb_market_data
     GROUP BY fund_code, market_date
     HAVING COUNT(1) > 1
""").df().head(5)

,count(1),FUND_CODE,MARKET_DATE,avg(daily_return)
0,2,FUND_01,2025-01-12,NaN
1,2,FUND_01,2025-01-27,0.210143
2,2,FUND_02,2025-01-17,0.234032
3,2,FUND_02,2025-01-30,0.245573
4,2,FUND_02,2025-02-28,0.223890


In [12]:
# fund_code + market_date + return_type is promissing because it appears to be unique
con.sql("""
    SELECT COUNT(1), fund_code, market_date, return_type, AVG(daily_return)
      FROM tb_market_data
     GROUP BY fund_code, market_date, return_type
     HAVING COUNT(1) > 1
""").df().head(5)

,count(1),FUND_CODE,MARKET_DATE,RETURN_TYPE,avg(daily_return)


In [13]:
# Creates the view used to display the daily and weekly returns
# This considers the business key as being fund_code + return_type + market_date
# This uses R = EXP(SUM(LN(1 + daily_return))) - 1 AS WEEKLY_RETURN to calculate the weekly return
con.execute("""
    CREATE OR REPLACE VIEW vw_mkt_returns AS
        WITH daily_returns AS (
            SELECT mkt.fund_code
                 , mkt.return_type
                 , mkt.market_date
                 , mkt.daily_return
                 , strftime('%Y-%W', mkt.market_date) AS YEAR_WEEK
              FROM tb_market_data AS mkt
              JOIN tb_calendar AS cal
                ON mkt.market_date = cal.date
             WHERE cal.is_working_day = TRUE
        )
      , weekly_returns AS (
            SELECT fund_code
                 , return_type
                 , year_week
                 , EXP(SUM(LN(1 + daily_return))) - 1 AS WEEKLY_RETURN
              FROM daily_returns
             GROUP BY fund_code
                    , return_type
                    , year_week
        )
        SELECT daily.fund_code
             , daily.return_type
             , daily.market_date
             , daily.daily_return
             , daily.year_week
             , weekly.weekly_return
          FROM daily_returns AS daily
          JOIN weekly_returns AS weekly
              ON daily.fund_code = weekly.fund_code
              AND daily.return_type = weekly.return_type
              AND daily.year_week = weekly.year_week
""")

In [14]:
# Display the View
con.sql("""
    SELECT * 
      FROM vw_mkt_returns
     ORDER BY fund_code
            , year_week
            , market_date
            , return_type
""")

┌───────────┬─────────────┬─────────────┬───────────────┬───────────┬────────────────────┐
│ FUND_CODE │ RETURN_TYPE │ MARKET_DATE │ DAILY_RETURN  │ YEAR_WEEK │   WEEKLY_RETURN    │
│  varchar  │   varchar   │    date     │    double     │  varchar  │       double       │
├───────────┼─────────────┼─────────────┼───────────────┼───────────┼────────────────────┤
│ FUND_01   │ TYPE_A      │ 2025-01-01  │  0.3007302582 │ 2025-00   │ 0.5582162431456255 │
│ FUND_01   │ TYPE_B      │ 2025-01-01  │ 0.08022171681 │ 2025-00   │ 0.5805392234716797 │
│ FUND_01   │ TYPE_A      │ 2025-01-02  │  0.1485102211 │ 2025-00   │ 0.5582162431456255 │
│ FUND_01   │ TYPE_B      │ 2025-01-02  │  0.3819792122 │ 2025-00   │ 0.5805392234716797 │
│ FUND_01   │ TYPE_A      │ 2025-01-03  │ 0.04305118198 │ 2025-00   │ 0.5582162431456255 │
│ FUND_01   │ TYPE_B      │ 2025-01-03  │ 0.05874375942 │ 2025-00   │ 0.5805392234716797 │
│ FUND_01   │ TYPE_A      │ 2025-01-06  │  0.1048321458 │ 2025-01   │ 0.9862496965157128 │

In [15]:
# Closes DuckDB Connection
con.close()

In [16]:
# Explanation of the Weekly Returns Calculation:

# This query creates a view that combines daily returns and weekly returns 
# for each fund and return type separately, matching the unique business key 
# for this dataset

# Step 1: daily_returns CTE
# - Filters only working days (markets open in Europe, 5 days/week)
# - Selects fund_code, return_type, market_date, and daily_return
# - Adds a YEAR_WEEK column to group dates by year and week

# Step 2: weekly_returns CTE
# - Calculates weekly return for each fund and return type using compounding:
#       weekly_return = EXP(SUM(LN(1 + daily_return))) - 1
#   This correctly compounds daily returns over the week
# - Groups by fund_code, return_type, and year_week

# Step 3: Final SELECT
# - Joins daily_returns and weekly_returns back together
# - Each daily row now shows:
#     - fund_code
#     - return_type
#     - market_date
#     - daily_return
#     - year_week
#     - weekly_return for that week and return type

# Observations:
# - The business key is fund_code + return_type + market_date because it ensures uniqueness
# - Weekly returns are calculated per return type separately 
# - If a fund has multiple return types, the total weekly return is NOT the sum 
#   of individual return types because compounding is multiplicative

# Main Difficulty:
# It took me some time to understand the formula and how to apply it in SQL

# Formula Explanation
# 1. (1+daily_return) The Growth Factor for a single day. The base amount plus the return
# 2. LN(…) Converts the multiplicative factor into an additive value. Takes the natural log of the growth factor
# 3. SUM(…) Aggregates the additive values across all days in the week. Calculates LN(1+r1)+LN(1+r2)+⋯+LN(1+rn)
# 4. EXP(…) Converts the sum back into a multiplicative value. Calculates eSUM(…), which equals ∏(1+ri)